In [6]:
# This is a script I applied early in toe competition ~ LB = 3. With bounding box regression  
# (applying this to fishes rather than whole images achieves a much better score)
# I used this script to learn about CNNs, feature extraction and using features learned by the InceptionV3 CNN
# to perform classificaiton using a SVM architecture.
# Inspired (adapted heavily) from: http://blog.christianperone.com/2015/08/convolutional-neural-networks-and-feature-extraction-with-python/

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import time

import os
import re

import tensorflow as tf
import tensorflow.python.platform
from tensorflow.python.platform import gfile
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import SVC, LinearSVC
import matplotlib.pyplot as plt
import pickle
from sklearn.calibration import CalibratedClassifierCV
from sklearn import svm

model_dir = 'model\\'
images_dir = 'training_images\\'

In [7]:
def main():
    list_images = []
    for path, subdirs, files in os.walk(images_dir):
        for name in files:
            if name.lower().endswith(".jpg"):
                list_images.append(path+"\\"+name)
            
    features,labels = extract_features(list_images)

    pickle.dump(features, open('features', 'wb'))
    pickle.dump(labels, open('labels', 'wb'))
    
    #features = pickle.load(open('features', 'rb'))
    #labels = pickle.load(open('labels', 'rb'))

    # run a 10-fold CV SVM using probabilistic outputs. 
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.1, random_state=0)
    clf = svm.SVC(kernel='linear', C=1).fit(X_train, y_train)
    clf.score(X_test, y_test)

    # probabalistic SVM
    clf =  CalibratedClassifierCV(clf, method='sigmoid', cv=2)
    clf.fit(X_train, y_train)
    y_pred = clf.predict_proba(X_test)
    
    k_fold = KFold(len(labels), shuffle=False, random_state=0)
    C_array=[0.001,0.01,0.1,1,10]
    C_scores=[]

    #C = 0.1 is best

    #clf = svm.LinearSVC(C=0.1)
    clf = svm.SVC(kernel='linear', C=0.1,probability=True)

    # final_model = clf.fit(features, labels)

    final_model = CalibratedClassifierCV(clf,cv=10,method='sigmoid')
    final_model = clf.fit(features, labels)

    test_dir='test_images\\'
    list_images = [test_dir+f for f in os.listdir(test_dir) if re.search('jpg|JPG', f)]
    
    features_test = extract_features2(list_images)

    y_pred = final_model.predict_proba(features_test)

    image_id = [i.split('\\')[-1] for i in list_images]#####

    submit = open('submit.SVM.csv','w')
    submit.write('image;BupInferior;BupSuperior;Formicidae_Inferior;Formicidae_Superior;MusInferior;MusSuperiorPentInferior;PentSuperior\n')

    for idx, id_n in enumerate(image_id):
        probs=['%s' % p for p in list(y_pred[idx, :])]
        submit.write('%s;%s\n' % (str(image_id[idx]),';'.join(probs)))

    submit.close()

In [8]:
# setup tensorFlow graph initiation
def create_graph():
    with gfile.FastGFile(os.path.join(model_dir, 'classify_image_graph_def.pb'), 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        _ = tf.import_graph_def(graph_def, name='')

In [9]:
# extract all features from pool layer of InceptionV3
def extract_features(list_images):
    nb_features = 2048
    features = np.empty((len(list_images),nb_features))
    labels = []
    create_graph()
    
    with tf.Session() as sess:
        next_to_last_tensor = sess.graph.get_tensor_by_name('pool_3:0')
        for ind, image in enumerate(list_images):
            print('Processing %s...' % (image))
            if not gfile.Exists(image):
                tf.logging.fatal('File does not exist %s', image)
            image_data = gfile.FastGFile(image, 'rb').read()    
            predictions = sess.run(next_to_last_tensor, {'DecodeJpeg/contents:0': image_data})
            features[ind,:] = np.squeeze(predictions)
            labels.append(image.split('\\')[1]+'_'+image.split('\\')[2])
        return features, labels

In [10]:
def extract_features2(list_images):
    nb_features = 2048
    features = np.empty((len(list_images),nb_features))
    create_graph()
    with tf.Session() as sess:
        next_to_last_tensor = sess.graph.get_tensor_by_name('pool_3:0')
        for ind, image in enumerate(list_images):
            print('Processing %s...' % (image))
            if not gfile.Exists(image):
                tf.logging.fatal('File does not exist %s', image)
            image_data = gfile.FastGFile(image, 'rb').read()
            predictions = sess.run(next_to_last_tensor, {'DecodeJpeg/contents:0': image_data})
            features[ind,:] = np.squeeze(predictions)
    return features

In [11]:
start = time.time()
main()
end = time.time()
print("Time: ",(end-start))
print("fim")

Processing training_images\Buprestidae\Inferior\0.jpg...
Processing training_images\Buprestidae\Inferior\1.jpg...
Processing training_images\Buprestidae\Inferior\10.jpg...
Processing training_images\Buprestidae\Inferior\100.jpg...
Processing training_images\Buprestidae\Inferior\101.jpg...
Processing training_images\Buprestidae\Inferior\102.jpg...
Processing training_images\Buprestidae\Inferior\103.jpg...
Processing training_images\Buprestidae\Inferior\104.jpg...
Processing training_images\Buprestidae\Inferior\105.jpg...
Processing training_images\Buprestidae\Inferior\106.jpg...
Processing training_images\Buprestidae\Inferior\107.jpg...
Processing training_images\Buprestidae\Inferior\108.jpg...
Processing training_images\Buprestidae\Inferior\109.jpg...
Processing training_images\Buprestidae\Inferior\11.jpg...
Processing training_images\Buprestidae\Inferior\110.jpg...
Processing training_images\Buprestidae\Inferior\111.jpg...
Processing training_images\Buprestidae\Inferior\112.jpg...
Pro

Processing training_images\Buprestidae\Inferior\36.jpg...
Processing training_images\Buprestidae\Inferior\37.jpg...
Processing training_images\Buprestidae\Inferior\38.jpg...
Processing training_images\Buprestidae\Inferior\39.jpg...
Processing training_images\Buprestidae\Inferior\4.jpg...
Processing training_images\Buprestidae\Inferior\40.jpg...
Processing training_images\Buprestidae\Inferior\41.jpg...
Processing training_images\Buprestidae\Inferior\42.jpg...
Processing training_images\Buprestidae\Inferior\43.jpg...
Processing training_images\Buprestidae\Inferior\44.jpg...
Processing training_images\Buprestidae\Inferior\45.jpg...
Processing training_images\Buprestidae\Inferior\46.jpg...
Processing training_images\Buprestidae\Inferior\47.jpg...
Processing training_images\Buprestidae\Inferior\48.jpg...
Processing training_images\Buprestidae\Inferior\49.jpg...
Processing training_images\Buprestidae\Inferior\5.jpg...
Processing training_images\Buprestidae\Inferior\50.jpg...
Processing train

Processing training_images\Buprestidae\Superior\162.jpg...
Processing training_images\Buprestidae\Superior\163.jpg...
Processing training_images\Buprestidae\Superior\164.jpg...
Processing training_images\Buprestidae\Superior\165.jpg...
Processing training_images\Buprestidae\Superior\166.jpg...
Processing training_images\Buprestidae\Superior\167.jpg...
Processing training_images\Buprestidae\Superior\168.jpg...
Processing training_images\Buprestidae\Superior\169.jpg...
Processing training_images\Buprestidae\Superior\17.jpg...
Processing training_images\Buprestidae\Superior\170.jpg...
Processing training_images\Buprestidae\Superior\171.jpg...
Processing training_images\Buprestidae\Superior\172.jpg...
Processing training_images\Buprestidae\Superior\173.jpg...
Processing training_images\Buprestidae\Superior\174.jpg...
Processing training_images\Buprestidae\Superior\175.jpg...
Processing training_images\Buprestidae\Superior\176.jpg...
Processing training_images\Buprestidae\Superior\177.jpg..

Processing training_images\Buprestidae\Superior\92.jpg...
Processing training_images\Buprestidae\Superior\93.jpg...
Processing training_images\Buprestidae\Superior\94.jpg...
Processing training_images\Buprestidae\Superior\95.jpg...
Processing training_images\Buprestidae\Superior\96.jpg...
Processing training_images\Buprestidae\Superior\97.jpg...
Processing training_images\Buprestidae\Superior\98.jpg...
Processing training_images\Buprestidae\Superior\99.jpg...
Processing training_images\Formicidae\Inferior\0.jpg...
Processing training_images\Formicidae\Inferior\1.jpg...
Processing training_images\Formicidae\Inferior\10.jpg...
Processing training_images\Formicidae\Inferior\100.jpg...
Processing training_images\Formicidae\Inferior\101.jpg...
Processing training_images\Formicidae\Inferior\102.jpg...
Processing training_images\Formicidae\Inferior\103.jpg...
Processing training_images\Formicidae\Inferior\104.jpg...
Processing training_images\Formicidae\Inferior\105.jpg...
Processing training

Processing training_images\Formicidae\Inferior\5.jpg...
Processing training_images\Formicidae\Inferior\50.jpg...
Processing training_images\Formicidae\Inferior\51.jpg...
Processing training_images\Formicidae\Inferior\52.jpg...
Processing training_images\Formicidae\Inferior\53.jpg...
Processing training_images\Formicidae\Inferior\54.jpg...
Processing training_images\Formicidae\Inferior\55.jpg...
Processing training_images\Formicidae\Inferior\56.jpg...
Processing training_images\Formicidae\Inferior\57.jpg...
Processing training_images\Formicidae\Inferior\58.jpg...
Processing training_images\Formicidae\Inferior\59.jpg...
Processing training_images\Formicidae\Inferior\6.jpg...
Processing training_images\Formicidae\Inferior\60.jpg...
Processing training_images\Formicidae\Inferior\61.jpg...
Processing training_images\Formicidae\Inferior\62.jpg...
Processing training_images\Formicidae\Inferior\63.jpg...
Processing training_images\Formicidae\Inferior\64.jpg...
Processing training_images\Formic

...

Processing training_images\Pentatomidae\Inferior\446.jpg...
Processing training_images\Pentatomidae\Inferior\447.jpg...
Processing training_images\Pentatomidae\Inferior\448.jpg...
Processing training_images\Pentatomidae\Inferior\449.jpg...
Processing training_images\Pentatomidae\Inferior\45.jpg...
Processing training_images\Pentatomidae\Inferior\450.jpg...
Processing training_images\Pentatomidae\Inferior\451.jpg...
Processing training_images\Pentatomidae\Inferior\452.jpg...
Processing training_images\Pentatomidae\Inferior\453.jpg...
Processing training_images\Pentatomidae\Inferior\454.jpg...
Processing training_images\Pentatomidae\Inferior\455.jpg...
Processing training_images\Pentatomidae\Inferior\456.jpg...
Processing training_images\Pentatomidae\Inferior\457.jpg...
Processing training_images\Pentatomidae\Inferior\458.jpg...
Processing training_images\Pentatomidae\Inferior\459.jpg...
Processing training_images\Pentatomidae\Inferior\46.jpg...
Processing training_images\Pentatomidae\In

Processing training_images\Pentatomidae\Superior\148.jpg...
Processing training_images\Pentatomidae\Superior\149.jpg...
Processing training_images\Pentatomidae\Superior\15.jpg...
Processing training_images\Pentatomidae\Superior\150.jpg...
Processing training_images\Pentatomidae\Superior\151.jpg...
Processing training_images\Pentatomidae\Superior\152.jpg...
Processing training_images\Pentatomidae\Superior\153.jpg...
Processing training_images\Pentatomidae\Superior\154.jpg...
Processing training_images\Pentatomidae\Superior\155.jpg...
Processing training_images\Pentatomidae\Superior\156.jpg...
Processing training_images\Pentatomidae\Superior\157.jpg...
Processing training_images\Pentatomidae\Superior\158.jpg...
Processing training_images\Pentatomidae\Superior\159.jpg...
Processing training_images\Pentatomidae\Superior\16.jpg...
Processing training_images\Pentatomidae\Superior\160.jpg...
Processing training_images\Pentatomidae\Superior\161.jpg...
Processing training_images\Pentatomidae\Su

Processing training_images\Pentatomidae\Superior\271.jpg...
Processing training_images\Pentatomidae\Superior\272.jpg...
Processing training_images\Pentatomidae\Superior\273.jpg...
Processing training_images\Pentatomidae\Superior\274.jpg...
Processing training_images\Pentatomidae\Superior\275.jpg...
Processing training_images\Pentatomidae\Superior\276.jpg...
Processing training_images\Pentatomidae\Superior\277.jpg...
Processing training_images\Pentatomidae\Superior\278.jpg...
Processing training_images\Pentatomidae\Superior\279.jpg...
Processing training_images\Pentatomidae\Superior\28.jpg...
Processing training_images\Pentatomidae\Superior\280.jpg...
Processing training_images\Pentatomidae\Superior\281.jpg...
Processing training_images\Pentatomidae\Superior\282.jpg...
Processing training_images\Pentatomidae\Superior\283.jpg...
Processing training_images\Pentatomidae\Superior\284.jpg...
Processing training_images\Pentatomidae\Superior\285.jpg...
Processing training_images\Pentatomidae\S

Processing training_images\Pentatomidae\Superior\395.jpg...
Processing training_images\Pentatomidae\Superior\396.jpg...
Processing training_images\Pentatomidae\Superior\397.jpg...
Processing training_images\Pentatomidae\Superior\398.jpg...
Processing training_images\Pentatomidae\Superior\399.jpg...
Processing training_images\Pentatomidae\Superior\4.jpg...
Processing training_images\Pentatomidae\Superior\40.jpg...
Processing training_images\Pentatomidae\Superior\400.jpg...
Processing training_images\Pentatomidae\Superior\401.jpg...
Processing training_images\Pentatomidae\Superior\402.jpg...
Processing training_images\Pentatomidae\Superior\403.jpg...
Processing training_images\Pentatomidae\Superior\404.jpg...
Processing training_images\Pentatomidae\Superior\405.jpg...
Processing training_images\Pentatomidae\Superior\406.jpg...
Processing training_images\Pentatomidae\Superior\407.jpg...
Processing training_images\Pentatomidae\Superior\408.jpg...
Processing training_images\Pentatomidae\Sup

Processing training_images\Pentatomidae\Superior\88.jpg...
Processing training_images\Pentatomidae\Superior\89.jpg...
Processing training_images\Pentatomidae\Superior\9.jpg...
Processing training_images\Pentatomidae\Superior\90.jpg...
Processing training_images\Pentatomidae\Superior\91.jpg...
Processing training_images\Pentatomidae\Superior\92.jpg...
Processing training_images\Pentatomidae\Superior\93.jpg...
Processing training_images\Pentatomidae\Superior\94.jpg...
Processing training_images\Pentatomidae\Superior\95.jpg...
Processing training_images\Pentatomidae\Superior\96.jpg...
Processing training_images\Pentatomidae\Superior\97.jpg...
Processing training_images\Pentatomidae\Superior\98.jpg...
Processing training_images\Pentatomidae\Superior\99.jpg...
Processing test_images\Buprestidae_Inferior-1.jpg...
Processing test_images\Buprestidae_Inferior-10.jpg...
Processing test_images\Buprestidae_Inferior-20.jpg...
Processing test_images\Buprestidae_Inferior-22.jpg...
Processing test_ima

In [ ]:
# This is a script I applied early in toe competition ~ LB = 3. With bounding box regression  
# (applying this to fishes rather than whole images achieves a much better score)
# I used this script to learn about CNNs, feature extraction and using features learned by the InceptionV3 CNN
# to perform classificaiton using a SVM architecture.
# Inspired (adapted heavily) from: http://blog.christianperone.com/2015/08/convolutional-neural-networks-and-feature-extraction-with-python/

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

#from subprocess import check_output
#print(check_output(["ls", "../input"]).decode("utf8"))

import os
import re

import tensorflow as tf
import tensorflow.python.platform
from tensorflow.python.platform import gfile
import numpy as np
import pandas as pd
import sklearn
#from sklearn import cross_validation
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import SVC, LinearSVC
import matplotlib.pyplot as plt
import pickle
from sklearn.calibration import CalibratedClassifierCV
from sklearn import svm

model_dir = 'model/'
# all training images
images_dir = 'training_images/'
list_images = [images_dir+f for f in os.listdir(images_dir) if re.search('jpg|JPG', f)]####

# setup tensorFlow graph initiation
def create_graph():
	with gfile.FastGFile(os.path.join(model_dir, 'classify_image_graph_def.pb'), 'rb') as f:
		graph_def = tf.GraphDef()
		graph_def.ParseFromString(f.read())
		_ = tf.import_graph_def(graph_def, name='')

# extract all features from pool layer of InceptionV3
def extract_features(list_images):
	nb_features = 2048
	features = np.empty((len(list_images),nb_features))
	labels = []
	create_graph()
	with tf.Session() as sess:
		next_to_last_tensor = sess.graph.get_tensor_by_name('pool_3:0')
		for ind, image in enumerate(list_images):
			print('Processing %s...' % (image))
			if not gfile.Exists(image):
				tf.logging.fatal('File does not exist %s', image)
			image_data = gfile.FastGFile(image, 'rb').read()
			predictions = sess.run(next_to_last_tensor,
			{'DecodeJpeg/contents:0': image_data})
			features[ind,:] = np.squeeze(predictions)
			labels.append(re.split('-\d+',image.split('/')[1])[0])#####
		return features, labels


features,labels = extract_features(list_images)

pickle.dump(features, open('features', 'wb'))
pickle.dump(labels, open('labels', 'wb'))

#features = pickle.load(open('features', 'rb'))
#labels = pickle.load(open('labels', 'rb'))

# run a 10-fold CV SVM using probabilistic outputs. 
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.1, random_state=0)
clf = svm.SVC(kernel='linear', C=1).fit(X_train, y_train)
clf.score(X_test, y_test)

# probabalistic SVM
clf =  CalibratedClassifierCV(clf, method='sigmoid', cv=2)
clf.fit(X_train, y_train)
y_pred = clf.predict_proba(X_test)

"""
### Funfou
clf = LinearSVC(C=1.0, loss='squared_hinge', penalty='l2',multi_class='ovr')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)


def plot_confusion_matrix(y_true,y_pred):
    cm_array = confusion_matrix(y_true,y_pred)
    true_labels = np.unique(y_true)
    pred_labels = np.unique(y_pred)
    plt.imshow(cm_array[:-1,:-1], interpolation='nearest', cmap=plt.cm.Blues)
    plt.title("Confusion matrix", fontsize=16)
    cbar = plt.colorbar(fraction=0.046, pad=0.04)
    cbar.set_label('Number of images', rotation=270, labelpad=30, fontsize=12)
    xtick_marks = np.arange(len(true_labels))
    ytick_marks = np.arange(len(pred_labels))
    plt.xticks(xtick_marks, true_labels, rotation=90)
    plt.yticks(ytick_marks,pred_labels)
    plt.tight_layout()
    plt.ylabel('True label', fontsize=14)
    plt.xlabel('Predicted label', fontsize=14)
    fig_size = plt.rcParams["figure.figsize"]
    fig_size[0] = 12
    fig_size[1] = 12
    plt.rcParams["figure.figsize"] = fig_size
    
print("Accuracy: {0:0.1f}%".format(accuracy_score(y_test,y_pred)*100))
plot_confusion_matrix(y_test,y_pred)
"""

k_fold = KFold(len(labels), shuffle=False, random_state=0)
C_array=[0.001,0.01,0.1,1,10]
C_scores=[]

"""
for k in C_array:
	clf = svm.SVC(kernel='linear', C=k)
	scores= cross_val_score(clf, features, labels, cv=k_fold, n_jobs=-1)
	C_scores.append(scores.mean())
	print(C_scores)
"""

#C = 0.1 is best

#clf = svm.LinearSVC(C=0.1)
clf = svm.SVC(kernel='linear', C=0.1,probability=True)

# final_model = clf.fit(features, labels)

final_model = CalibratedClassifierCV(clf,cv=10,method='sigmoid')
final_model = clf.fit(features, labels)


test_dir='test_images/'
list_images = [test_dir+f for f in os.listdir(test_dir) if re.search('jpg|JPG', f)]#####


def extract_features(list_images):
	nb_features = 2048
	features = np.empty((len(list_images),nb_features))
	create_graph()
	with tf.Session() as sess:
		next_to_last_tensor = sess.graph.get_tensor_by_name('pool_3:0')
		for ind, image in enumerate(list_images):
			print('Processing %s...' % (image))
			if not gfile.Exists(image):
				tf.logging.fatal('File does not exist %s', image)
			image_data = gfile.FastGFile(image, 'rb').read()
			predictions = sess.run(next_to_last_tensor,
			{'DecodeJpeg/contents:0': image_data})
			features[ind,:] = np.squeeze(predictions)
		return features


features_test = extract_features(list_images)

y_pred = final_model.predict_proba(features_test)
#y_pred = final_model.predict(features_test)
#y_pred = final_model.predict(features_test)
    
image_id = [i.split('/')[-1] for i in list_images]#####

submit = open('submit.SVM.csv','w')
submit.write('image;BupInferior;BupSuperior;Formicidae_Inferior;Formicidae_Superior;PentInferior;PentSuperior\n')

for idx, id_n in enumerate(image_id):
	probs=['%s' % p for p in list(y_pred[idx, :])]
	submit.write('%s;%s\n' % (str(image_id[idx]),';'.join(probs)))####

submit.close()
print("FIM")

In [ ]:
print(clf)

In [ ]:
'''
# setup tensorFlow graph initiation
def create_graph():
	with gfile.FastGFile(os.path.join(model_dir, 'output.pb'), 'rb') as f:
		graph_def = tf.GraphDef()
		graph_def.ParseFromString(f.read())
		_ = tf.import_graph_def(graph_def, name='')

# extract all features from pool layer of InceptionV3
def extract_features(list_images):
	nb_features = 2048
	features = np.empty((len(list_images),nb_features))
	labels = []
	create_graph()
	with tf.Session() as sess:
		next_to_last_tensor = sess.graph.get_tensor_by_name('pool_3:0')
		for ind, image in enumerate(list_images):
			print('Processing %s...' % (image))
			if not gfile.Exists(image):
				tf.logging.fatal('File does not exist %s', image)
			image_data = gfile.FastGFile(image, 'rb').read()
			predictions = sess.run(next_to_last_tensor,
			{'DecodeJpeg/contents:0': image_data})
			features[ind,:] = np.squeeze(predictions)
			labels.append(re.split('_\d+',image.split('/')[1])[0])
		return features, labels


features,labels = extract_features(list_images)

pickle.dump(features, open('features', 'wb'))
pickle.dump(labels, open('labels', 'wb'))

features = pickle.load(open('features'))
labels = pickle.load(open('labels'))

# run a 10-fold CV SVM using probabilistic outputs. 
X_train, X_test, y_train, y_test = cross_validation.train_test_split(features, labels, test_size=0.1, random_state=0)
clf = svm.SVC(kernel='linear', C=1).fit(X_train, y_train)
clf.score(X_test, y_test)

# probabalistic SVM
clf =  sklearn.calibration.CalibratedClassifierCV(svm)
clf.fit(X_train, y_train)
y_pred = clf.predict_proba(X_test)


k_fold = KFold(len(labels),n_folds=10, shuffle=False, random_state=0)
C_array=[0.001,0.01,0.1,1,10]
C_scores=[]

for k in C_array:
	clf = svm.SVC(kernel='linear', C=k)
	scores= cross_val_score(clf, features, labels, cv=k_fold, n_jobs=-1)
	C_scores.append(scores.mean())
	print C_scores

#C = 0.1 is best

#clf = svm.LinearSVC(C=0.1)
clf = svm.SVC(kernel='linear', C=0.1,probability=True)

# final_model = clf.fit(features, labels)

final_model = CalibratedClassifierCV(clf,cv=10,method='sigmoid')
final_model = clf.fit(features, labels)


test_dir='latest_submission/test_stg1/test_stg1/'
list_images = [test_dir+f for f in os.listdir(test_dir) if re.search('jpg|JPG', f)]


def extract_features(list_images):
	nb_features = 2048
	features = np.empty((len(list_images),nb_features))
	create_graph()
	with tf.Session() as sess:
		next_to_last_tensor = sess.graph.get_tensor_by_name('pool_3:0')
		for ind, image in enumerate(list_images):
			print('Processing %s...' % (image))
			if not gfile.Exists(image):
				tf.logging.fatal('File does not exist %s', image)
			image_data = gfile.FastGFile(image, 'rb').read()
			predictions = sess.run(next_to_last_tensor,
			{'DecodeJpeg/contents:0': image_data})
			features[ind,:] = np.squeeze(predictions)
		return features


features_test = extract_features(list_images)

y_pred = final_model.predict_proba(features_test)
#y_pred = final_model.predict(features_test)
#y_pred = final_model.predict(features_test)


image_id = [i.split('/')[3] for i in list_images]

submit = open('submit.SVM.csv','w')
submit.write('image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT\n')

for idx, id_n in enumerate(image_id):
	probs=['%s' % p for p in list(y_pred[idx, :])]
	submit.write('%s,%s\n' % (str(image_id[idx]),','.join(probs)))

submit.close()
'''